In [ ]:
# This notebook records related code for the 3 million spectral pairs derived from 2571 curated spectra
# Here is a diagram showing the relative paths of the input and output files and folders
# We provide compressed files in three folders: notebook, src, and msdb, at https://zenodo.org/uploads/17065209
# Most input and output files are included except for some very large files, exceeding 20 GB, like the pairwise similarity matrix of each in-silico library.
# Before using these codes, make sure to use them in the msanalyst folder cloned from git.
'''
MSanalyst/
    ├── notebook/
    ├── src/
    └── msdb/
        ├── GNPSLIBRARY/
        |   ├── ALL_GNPS_NO_PROPOGATED.mgf  (input)
        |   ├── GNPS-LIBRARY.mgf  (input)
        |   ├── edbMS1.csv  (output)
        |   ├── edb_info.json  (output)
        |   ├── GNPS-LIBRARY-INFO.json  (output)
        |   ├── GNPS-LIBRARY-INFO_new.json  (output)
        |   └── FS_edb_smi.json  (input)
        ├── GNPSLIBRARY_250514/
        |   └── ALL_GNPS_NO_PROPOGATED.mgf  (input)
        ├── COCONUT_2506/
        |   ├── coconut_csv_lite-06-2025.csv  (input)
        |   └── COCONUT.json  (output)
        ├── data/
        |   ├── hqtof/
        |   |   ├── idlist/
        |   |   |   └── H_qtof_non-redundant_CCMSIDs.npy  (output)
        |   |   └── FS_hqtof.json  (output)
        |   └── idlist/
        |       ├── NpclasstoSupplement.npy  (output)
        |       ├── NpclasstoSupplement.csv  (output)
        |       └── NpclasstoSupplement.json  (output)
        ├── FS_isdb_e0.json  (output)
        ├── FS_isdb_e1.json  (output)
        ├── FS_isdb_e2.json  (output)
        └── isdb_info.json  (input)
'''

# Import library 

In [1]:
import sys,os,json,spectral_entropy,sys,time
sys.argv = ['jupyter']  # Clear the parameters passed to Jupyter
sys.path.append('../')
import pandas as pd
import numpy as np
import matplotlib.ticker as mticker
import seaborn as sns
import matplotlib.pyplot as plt
import spectrum_utils.spectrum as sus
from spectral_entropy.spectral_entropy import calculate_entropy
from my_packages import peaktools, functions_new, cheminfo_tools, evaluation
from matplotlib.colors import LogNorm
from matchms.importing import load_from_mgf
from tqdm import tqdm, trange
from ms_entropy import FlashEntropySearch
from sklearn.metrics import roc_curve,auc,confusion_matrix
from FPSim2.io import create_db_file
from FPSim2 import FPSim2Engine
from rdkit import DataStructs
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.decomposition import PCA
from matchms import Spectrum
from matchms.similarity import ModifiedCosine,NeutralLossesCosine,CosineGreedy

Determination of memory status is not supported on this 
 platform, measuring for memoryleaks will never fail


# Load library files
## EDB

In [ ]:
# Load GNPS library file
t = time.time() 

# GNPS_FILE = '../msdb/GNPSLIBRARY/ALL_GNPS_NO_PROPOGATED.mgf'
GNPS_CLEAN_FILE = '../msdb/GNPSLIBRARY/ALL_GNPS_cleaned.mgf'
# GNPS_DEMO_FILE = '../msdb/GNPSLIBRARY/GNPS-LIBRARY-demo.mgf'

# GNPS_INFO = list(load_from_mgf(GNPS_FILE))
GNPS_CLAEN_INFO = list(load_from_mgf(GNPS_CLEAN_FILE))
print(f'Finished in {(time.time() - t) / 60:.2f} min')

In [ ]:
# Load library spectral file
t = time.time()

# ISMS1_CSV = '../msdb/isdbMS1.csv' # 'id', 'formula', 'exactmass', 'smiles', 'inchi', 'inchikey', 'm+h','m+nh4', 'm+na'
# ISMS1_DF = pd.read_csv(ISMS1_CSV,low_memory=False)
# EMS1_CSV = '../msdb/edbMS1.csv' # 'id', 'pepmass', 'smiles'
# EMS1_DF = pd.read_csv(EMS1_CSV)

# Flash search - Load
FS_E_LIBRARY = functions_new.json_load('../msdb/FS_edb_info.json')
e_search = FlashEntropySearch()
FS_E_LIBRARY = e_search.build_index(FS_E_LIBRARY)

print(f'Finished in {(time.time() - t) / 60:.2f} min')

## ISDB

In [ ]:
# ISDB energy 10,20,40
ISDB_INFO = functions_new.json_load('../msdb/isdb_info.json')

In [ ]:
# ISDB energy = 10
t = time.time()
FS_IS_LIBRARY_e0 = functions_new.json_load('../msdb/FS_isdb_e0.json')
is0_search = FlashEntropySearch()
FS_IS0_LIBRARY = is0_search.build_index(FS_IS_LIBRARY_e0)

# ISDB energy = 20
t = time.time()
FS_IS_LIBRARY_e1 = functions_new.json_load('../msdb/FS_isdb_e1.json')
is1_search = FlashEntropySearch()
FS_IS1_LIBRARY = is1_search.build_index(FS_IS_LIBRARY_e1)

# ISDB energy = 40
t = time.time()
FS_IS_LIBRARY_e2 = functions_new.json_load('../msdb/FS_isdb_e2.json')
is2_search = FlashEntropySearch()
FS_IS2_LIBRARY = is2_search.build_index(FS_IS_LIBRARY_e2)
print(f'Finished in {(time.time() - t) / 60:.2f} min')

## Information supplement

In [ ]:
# Supplement the library with additional information (e.g., entropy, n_peaks, classes)
# Param: FS_E_LIBRARY,FS_IS0_LIBRARY,FS_IS1_LIBRARY,FS_IS2_LIBRARY,
# Return: LIBRARIES with supplemented information 
for SPECTRUM in tqdm(FS_E_LIBRARY, total = len(FS_E_LIBRARY)):
    PEAKs = SPECTRUM['peaks']
    SPECTRUM['n_peak'] = len(PEAKs)
    SPECTRUM['entropy'] = calculate_entropy(PEAKs)
    

In [ ]:
for SPECTRUM in tqdm(FS_IS0_LIBRARY, total = len(FS_IS0_LIBRARY)):
    PEAKs = SPECTRUM['peaks']
    SPECTRUM['n_peak'] = len(PEAKs)
    SPECTRUM['entropy'] = calculate_entropy(PEAKs)
    SPECTRUM['peaks'] = PEAKs.tolist()
    
for SPECTRUM in tqdm(FS_IS1_LIBRARY, total = len(FS_IS1_LIBRARY)):
    PEAKs = SPECTRUM['peaks']
    SPECTRUM['n_peak'] = len(PEAKs)
    SPECTRUM['entropy'] = calculate_entropy(PEAKs)
    SPECTRUM['peaks'] = PEAKs.tolist()

for SPECTRUM in tqdm(FS_IS2_LIBRARY, total = len(FS_IS2_LIBRARY)):
    PEAKs = SPECTRUM['peaks']
    SPECTRUM['n_peak'] = len(PEAKs)
    SPECTRUM['entropy'] = calculate_entropy(PEAKs)
    SPECTRUM['peaks'] = PEAKs.tolist()

In [ ]:
with open('../msdb/FS_isdb_e0.json', "w") as f:
    json.dump(FS_IS0_LIBRARY, f)

with open('../msdb/FS_isdb_e1.json', "w") as f:
    json.dump(FS_IS1_LIBRARY, f)

with open('../msdb/FS_isdb_e2.json', "w") as f:
    json.dump(FS_IS2_LIBRARY, f)

## Hqtof lib Data for Figure 3H-I

  
Since spectra have different adduct types,sourced from redundant structures and different instrument platforms
It is necessary to remove the influences mentioned above before evaluating the spectral similarity matrices

In [ ]:
# Create hqtof data
GNPS_SPECTRA = functions_new.load_spectra_from_file('../msdb/GNPSLIBRARY/GNPS-LIBRARY.mgf')
Hqtof_CCMSIDs = list(np.load('../msdb/data/hqtof/idlist/H_qtof_non-redundant_CCMSIDs.npy'))

FS_SPECTRA = []
for SPECTRUM in tqdm(GNPS_SPECTRA,total = len(GNPS_SPECTRA)):
    CCMSID = SPECTRUM.metadata['spectrum_id']
    if CCMSID in Hqtof_CCMSIDs:
        PM = SPECTRUM.metadata['precursor_mz']
        IONMODE = SPECTRUM.metadata['ionmode']
        try:
            SMILE = SPECTRUM.metadata['smiles']
        except:
            SMILE = GNPS_INFO[CCMSID]['CANONSMILES']
            
        CHARGE = SPECTRUM.metadata['charge']
        SCLASS = GNPS_INFO[CCMSID]['np_classifier_superclass']
        
        peaks = np.column_stack((SPECTRUM.mz,SPECTRUM.intensities))
        # peaks = se.clean_spectrum(peaks, max_mz=PM + 1.6)
        FS_SPECTRA.append({
            "id": CCMSID,
            "precursor_mz": PM,
            "peaks": peaks.tolist(),
            "smile": SMILE,
            "charge": CHARGE,
            "ion_mode":IONMODE,
            'superclass':[SCLASS]
            })

# Save
sorted_list = sorted(FS_SPECTRA, key=lambda x: x['precursor_mz'])
FS_GNPS_LIBRARY_OUTPUT = '../msdb/data/hqtof/FS_hqtof.json'
with open(FS_GNPS_LIBRARY_OUTPUT, "w") as f:
    json.dump(sorted_list, f)

In [24]:
# Load hqtof data
HQTOF = functions_new.json_load('../msdb/data/hqtof/FS_hqtof.json')
# hqtof_search = FlashEntropySearch()
# FS_SPECTRA = hqtof_search.build_index(HQTOF) # Pre-clean and sorted flash spectra

2571

# Figure S1 Kdeplot

In [ ]:
width = 3.5
height = width / 1.618
# To take an overview of the ISDB and EDB
# Param: FS_E_LIBRARY,FS_IS0_LIBRARY,FS_IS1_LIBRARY,FS_IS2_LIBRARY,
# Return: kdeplot

## EDB

In [ ]:
t = time.time()
E_pm,E_npeak,E_entropy = [],[],[]
for SPECTRUM in tqdm(FS_E_LIBRARY,total = len(FS_E_LIBRARY)):
    E_pm.append(SPECTRUM['precursor_mz'])
    E_npeak.append(SPECTRUM['n_peak'])
    E_entropy.append(SPECTRUM['entropy'])
E_DF = pd.DataFrame({
    'entropy': E_entropy,
    'PM': E_pm,
    'NPEAK':E_npeak
})

In [ ]:
# Random sample
sample_size = 50000  # Randomly select one-tenth of the entire library
sample_df = E_DF.sample(n=sample_size)

plt.figure(figsize=(width * 2, height * 2))
kde = sns.kdeplot(
    data=sample_df,
    x='entropy',
    y='PM',
    fill=True,
    common_grid=True,
    common_norm=False,
    bw_adjust=2,
    # cut=0,
    cbar=True,
    legend=False,
    color = '#E84445'
)
plt.ylim(0, 1500)
plt.xlim(0, 6)
plt.savefig(f'../msdb/data/pic/E_distribution.png', dpi=300, bbox_inches= 'tight')

## ISDB_e0

In [ ]:
# ISDB_e0
t = time.time()
IS0_pm,IS0_npeak,IS0_entropy = [],[],[]
for SPECTRUM in tqdm(FS_IS0_LIBRARY,total = len(FS_IS0_LIBRARY)):
    IS0_pm.append(SPECTRUM['precursor_mz'])
    IS0_npeak.append(SPECTRUM['n_peak'])
    IS0_entropy.append(SPECTRUM['entropy'])
IS0_DF = pd.DataFrame({
    'entropy': IS0_entropy,
    'PM': IS0_pm,
    'NPEAK':IS0_npeak
})

print(f'Finished in {(time.time() - t) / 60:.2f} min')

In [ ]:
sample_size = 50000  # Randomly select one-tenth of the entire library
i0_df = IS0_DF.sample(n=sample_size, random_state=42)
width = 3.5
height = width / 1.618
plt.figure(figsize=(width*2 , height*2))
kde = sns.kdeplot(
    data=i0_df,
    x='entropy',
    y='PM',
    fill=True,
    common_grid=True,
    common_norm=False,
    bw_adjust=2,
    cbar=True,
    # cut=0,
    color = '#1999B2'
)

plt.ylim(0, 1500)
plt.xlim(0, 6)
plt.savefig(f'../msdb/data/pic/is0_distribution.png', dpi=300, bbox_inches= 'tight')

## ISDB_e1

In [ ]:
# ISDB_e1
t = time.time()
IS1_pm,IS1_npeak,IS1_entropy = [],[],[]
for SPECTRUM in tqdm(FS_IS1_LIBRARY,total = len(FS_IS1_LIBRARY)):
    IS1_pm.append(SPECTRUM['precursor_mz'])
    IS1_npeak.append(SPECTRUM['n_peak'])
    IS1_entropy.append(SPECTRUM['entropy'])
IS1_DF = pd.DataFrame({
    'entropy': IS1_entropy,
    'PM': IS1_pm,
    'NPEAK':IS1_npeak
})


In [ ]:
sample_size = 50000  # Randomly select one-tenth of the entire library
i1_df = IS1_DF.sample(n=sample_size)

plt.figure(figsize=(width*2 , height*2))
kde = sns.kdeplot(
    data=i1_df,
    x='entropy',
    y='PM',
    fill=True,
    common_grid=True,
    common_norm=False,
    bw_adjust=2,
    # cut=0,
    cbar=True,
    legend=False,
    color = '#95BCE5'
)
plt.ylim(0, 1500)
plt.xlim(0, 6)
plt.savefig(f'../msdb/data/pic/is1_distribution.png', dpi=300, bbox_inches= 'tight')

## ISDB_e2

In [ ]:
# ISDB_e2
t = time.time()
IS2_pm,IS2_npeak,IS2_entropy = [],[],[]
for SPECTRUM in tqdm(FS_IS2_LIBRARY,total = len(FS_IS2_LIBRARY)):
    IS2_pm.append(SPECTRUM['precursor_mz'])
    IS2_npeak.append(SPECTRUM['n_peak'])
    IS2_entropy.append(SPECTRUM['entropy'])
IS2_DF = pd.DataFrame({
    'entropy': IS2_entropy,
    'PM': IS2_pm,
    'NPEAK':IS2_npeak
})

In [ ]:
sample_size = 50000  # Randomly select one-tenth of the entire library
i2_df = IS2_DF.sample(n=sample_size)

plt.figure(figsize=(width*2 , height*2))
kde = sns.kdeplot(
    data=i2_df,
    x='entropy',
    y='PM',
    fill=True,
    common_grid=True,
    common_norm=False,
    bw_adjust=2,
    # cut=0,
    cbar=True,
    legend=False,
    color = '#F39DA0'
)
plt.ylim(0, 1500)
plt.xlim(0, 6)
plt.savefig(f'../msdb/data/pic/is2_distribution.png', dpi=300, bbox_inches= 'tight')

# R3Q14 Accuracy of ISDB
##  Inner consistency of ISDB
To answer Review3 comment 14
### Pairwise Spectral similarity of ISDB_e0/e1/e2

In [ ]:
# The pairwise similarity is calculated and saved using Scripts: "../src/ICofIS.py"

In [ ]:
# Id corresponds to Smile
icis0 = functions_new.json_load('../msdb/data/ICofIS/is0_Id2Smile.json')
icis1 = functions_new.json_load('../msdb/data/ICofIS/is1_Id2Smile.json')
icis2 = functions_new.json_load('../msdb/data/ICofIS/is2_Id2Smile.json')

### Pairwise chemical similarity matrices

In [ ]:
# The structural similarity can be calculated by "../src/ICofIS_ChemSim.py"  or the following code
# Create fp database by FPsim2
# Param: Index-to-SMILE, specific finger print type(Morgan_radius2_fpSize1024)
# return: fp database file
t = time.time() 
EL = 2 # 0, 1 Energy Lever
ID2SMILE = functions_new.json_load(f'../msdb/data/ICofIS/is{EL}_Id2Smile.json')
FPDB_FILE = f'../msdb/data/ICofIS/chem/is{EL}_fp_db.h5'
OUTPUT_MATRIX = f'../msdb/data/ICofIS/chem/is{EL}_ICofIS.npy'

IDX2SMILE = [[s['smile'],idx] for idx,s in enumerate(ID2SMILE)]

if not os.path.exists(FPDB_FILE):
    create_db_file(
        mols_source=IDX2SMILE, 
        filename=FPDB_FILE,
        mol_format='smiles', # required
        fp_type='Morgan',
        fp_params={'radius': 2, 'fpSize': 1024}
    )
print(f'Finished in {(time.time() - t) / 60:.2f} min')

# Calculate structural similarity pair >= specific threshold (Dice_radius2_0.75)
fpe = FPSim2Engine(FPDB_FILE)
hit_dict = {}
for idx,s in tqdm(enumerate(IDX2SMILE),total = len(IDX2SMILE)):
    SMILE = s[0]
    results = fpe.similarity(SMILE, threshold=0.75, metric='dice', n_workers=1)# [(LIBidx,similarity1),(),...]
    fpidx_list = [x[0] for x in results]
    fpsim_list = [x[1] for x in results]
    if fpidx_list:
        hit_dict[idx] = {'idx':fpidx_list,'sim_list':fpsim_list} # {LIBidx: {},{},..}

# Save matrix
ECHEM_MATRIX  = np.full((len(IDX2SMILE),len(IDX2SMILE)), 0.0, dtype=float) 
for key, value in hit_dict.items():
    QueryIdx = key
    MatchIdxs = value['idx']
    MatchSim = value['sim_list']
    for idx, MatchIdx in enumerate(MatchIdxs):
        ECHEM_MATRIX[QueryIdx,MatchIdx] = MatchSim[idx]
np.save(OUTPUT_MATRIX,ECHEM_MATRIX)


### Molecular networking 

In [3]:
# Filtering edges based on the spectral similarity matrix generated above to create MN using "../src/ICofIS_networking.py"

### Evaluation

In [ ]:
# Evaluating the generated network using "../src/ICofIS_evaluation.py"

### PCA plot

In [ ]:
def fp2arr(fp):
    arr = np.zeros((1,))
    DataStructs.ConvertToNumpyArray(fp, arr)
    return arr

In [ ]:
# small sample represents the entire library
# icis0, icis1, icis2 <dict> FS_IS0_LIBRARY
class1=[]
molecules=[]
ecfp_1024=[]
X = []
CLASS1 = 'class1'
CLASS2 = 'class2'
for i in trange(len(icis0)):
    try:
        SMILE = icis0[i]['smile']
        mol = Chem.MolFromSmiles(SMILE)
        ecfp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=1024)
        class1.append(CLASS1)
        X.append(np.array(fp2arr(ecfp)))
    except:
        pass

for i in trange(len(FS_IS0_LIBRARY)):
    try:
        SMILE = FS_IS0_LIBRARY[i]['smile']
        mol = Chem.MolFromSmiles(SMILE)
        ecfp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=1024)
        class1.append(CLASS2)
        X.append(np.array(fp2arr(ecfp)))
    except:
        pass


pca = PCA()
pca.fit(X)
reduced_X = pca.transform(X)

# Access the explained variance ratio of each principal component
explained_var_ratio = pca.explained_variance_ratio_

# Print the explained variance ratio of each principal component
pc1 = ''
pc2 = ''
pc3 = ''
for i, var_ratio in enumerate(explained_var_ratio):
    print(f'Principal Component {i + 1}: Explained Variance Ratio = {var_ratio:.4f}')
    if i+1 == 1:
        pc1 = f'{(var_ratio*100):.2f}%'
    if i+1 == 2:
        pc2 = f'{(var_ratio*100):.2f}%'
    if i+1 == 3:
        pc3 = f'{(var_ratio*100):.2f}%'
    if i+1 > 2:break

print(pc1,pc2,pc3)
red_x, red_y, red_z= [], [], []
blue_x, blue_y, blue_z = [], [], []
green_x, green_y, green_z = [], [], []

for i in trange(len(reduced_X)):
    if class1[i] == CLASS1:
        red_x.append(reduced_X[i][0])
        red_y.append(reduced_X[i][1])

    elif class1[i] == CLASS2:
        blue_x.append(reduced_X[i][0])
        blue_y.append(reduced_X[i][1])
  


fig = plt.figure(dpi=150)
ax = fig.add_subplot(111)
ax.scatter(red_x, red_y, c='#1A51AD', marker='o', label='selected',alpha=0.5)
ax.scatter(blue_x, blue_y, c='gray', marker='o', label='ISDB',alpha=0.015)
ax.legend()
ax.set_xlabel(f'PC1 ({pc1})')
ax.set_ylabel(f'PC2 ({pc2})')
ax.set_title('2D PCA')

plt.show()

## Similarity between in-silico and experimental spectra

In [ ]:
# Duplicate
temp = functions_new.json_load('../msdb/FS_edb_smi.json')
e_search = FlashEntropySearch()
FS_E_LIBRARY = e_search.build_index(temp)

In [111]:
# Retain spectra with non-redunant similes in experimental library
dup_E_LIBRARY = list({s['smile']: s for s in FS_E_LIBRARY}.values()) 
ISDB_list = list(ISDB_INFO) # Keys of ISDB_INFO <list>

In [ ]:
# gnps_is:h qtof (47192, 467124)
# Create fp databased by FPsim2
# Param: Index-to-SMILE, specific finger print type(Morgan_radius2_fpSize1024)
# return: fp database file
t = time.time() 
DIR = f'../msdb/data/ICofIS/'
FPDB_FILE = os.path.join(DIR,f'chem/fp_db_gnps_is.h5')
OUTPUT_MATRIX = os.path.join(DIR,f'chem/gnps_is.npy')

IDX2SMILE = []
for idx, (key, values) in enumerate(ISDB_INFO.items()):
    IDX2SMILE.append([values['smiles'],idx])

if not os.path.exists(FPDB_FILE):
    create_db_file(
        mols_source=IDX2SMILE, 
        filename=FPDB_FILE,
        mol_format='smiles', # required
        fp_type='Morgan',
        fp_params={'radius': 2, 'fpSize': 1024}
    )
print(f'Finished in {(time.time() - t) / 60:.2f} min')


In [84]:
# Find identical GNPS-smile in isdb
# Calculate structural similarity pair >= specific threshold (Dice_radius2_1)
# FS_SPECTRA: hqtof
fpe = FPSim2Engine(FPDB_FILE)
hit_dict = {}
for idx,s in enumerate(dup_E_LIBRARY):
    SMILE = s['smile']
    results = fpe.similarity(SMILE, threshold=1, metric='dice', n_workers=1)# [(LIBidx,similarity1),(),...]
    fpidx_list = [x[0] for x in results]
    fpsim_list = [x[1] for x in results]
    if fpidx_list:
        hit_dict[idx] = {'idx':fpidx_list,'sim_list':fpsim_list} # {LIBidx: {},{},..}

In [120]:
# 18983 gnps-smiles found identical match in isdb
ALGO = 'entropy'
for key, values in hit_dict.items():
    dup_E_SPECTRUM = dup_E_LIBRARY[key]
    PM = dup_E_SPECTRUM['precursor_mz']
    SPEC = dup_E_SPECTRUM['peaks']
    SPEC = spectral_entropy.clean_spectrum(SPEC, max_mz=PM+0.01) # MS2 spectrum clean by normalizing and removing signals with intensity less than 1% of the base peak
    SPECTRUM = Spectrum(mz=np.array(SPEC[:, 0],dtype = float),
                  intensities=np.array(SPEC[:, 1],dtype = float),
                  metadata={"precursor_mz": PM})
    
    
    ISDB_idx = values['idx'][0]
    ISDB_ID = ISDB_list[ISDB_idx]
    e0_ms2,e0_spectrum,e1_ms2,e1_spectrum,e2_ms2,e2_spectrum = functions_new.get_spectra_from_isinfo(ISDB_INFO,ISDB_ID)

    SIM0, MP0 = functions_new.clac_spec_sim(SPEC,SPECTRUM,e0_ms2,e0_spectrum,ALGO)
    SIM1, MP1 = functions_new.clac_spec_sim(SPEC,SPECTRUM,e1_ms2,e1_spectrum,ALGO)
    SIM2, MP2 = functions_new.clac_spec_sim(SPEC,SPECTRUM,e2_ms2,e2_spectrum,ALGO)

    dup_E_SPECTRUM[f'{ALGO}_sim0'] = round(SIM0,2)
    dup_E_SPECTRUM[f'{ALGO}_mp0'] = MP0
    dup_E_SPECTRUM[f'{ALGO}_sim1'] = round(SIM1,2)
    dup_E_SPECTRUM[f'{ALGO}_mp1'] = MP1
    dup_E_SPECTRUM[f'{ALGO}_sim2'] = round(SIM2,2)
    dup_E_SPECTRUM[f'{ALGO}_mp2'] = MP2

In [131]:
dup_E_LIBRARY[19]
for r in dup_E_LIBRARY:
    print(r)
    break

{'id': 'CCMSLIB00000427077', 'precursor_mz': 32.05, 'peaks': array([[30.,  1.]], dtype=float32), 'smile': 'CN', 'charge': 1, 'ion_mode': 'positive'}


In [203]:
ALGO = 'peak_percentage'
keys = [f'{ALGO}_sim0',f'{ALGO}_mp0',
        f'{ALGO}_sim1',f'{ALGO}_mp1',
        f'{ALGO}_sim2',f'{ALGO}_mp2']
stats = {keys[0]:[],keys[1]:[],keys[2]:[],keys[3]:[],keys[4]:[],keys[5]:[]}
for SPECTRUM in dup_E_LIBRARY:
    for key in keys:
        try:
            stats[key].append(SPECTRUM[key])
        except:
            pass
idx1,idx3,idx5 = 0,2,4
total = len(stats[keys[idx1]])
print(f'{len([x for x in stats[keys[idx5]] if x >= 0.6])/total}% pairs higher than 0.6')
print(f'{len([x for x in stats[keys[idx1]] if 0.7 > x >= 0.35])/total}% pairs higher than 0.35 and lower than 0.6')
print(f'{len([x for x in stats[keys[idx1]] if 0.35 > x])/total}% pairs lower than 0.35')

0.09993151767370806% pairs higher than 0.6
0.276563240794395% pairs higher than 0.35 and lower than 0.6
0.6691250065848391% pairs lower than 0.35


In [194]:
# Plotting
plt.rcParams['font.sans-serif'] = ['Arial'] 
fig, ax1 = plt.subplots(figsize=(3, 6))
idx1,idx2 = 4,5
x_ls = np.zeros(len(stats[keys[idx1]]))     
x_hs = np.ones(len(stats[keys[idx2]]))      

# Violin plot of low similarity pairs
# sns.violinplot(y=stats[keys[0]], color='blue', inner=None, ax=ax1, width=0.5)
sns.violinplot(x=x_ls, y=stats[keys[idx1]],
               color='#F5A889', inner=None, width=0.4, ax=ax1)
ax1.set_title(f"{keys[idx1]}")
ax1.set_ylabel("Spectral similarity", color='black')
ax1.tick_params(axis='y', labelcolor='black')

ax1.xaxis.set_ticks_position('none') 
ax1.yaxis.set_ticks_position('none')  

ax2 = ax1.twinx() 
ax2.spines['right'].set_position(('outward', -3)) 

# Violin plot of high similarity pairs
sns.violinplot(x=x_hs, y=stats[keys[idx2]],
               color='#ACD6EC', inner=None, width=0.4, ax=ax2)
ax2.set_ylabel("Shared peaks", color='black')
ax2.tick_params(axis='y', labelcolor='black')


plt.tight_layout()
plt.savefig(os.path.join(DIR,f'pic/violion_{keys[idx1]}.png'), dpi=500, bbox_inches="tight")
# plt.show()
plt.close()

# Figure 2G-I Comparative analysis
Calculating pairwise spectral and structural similarity based on the curated hqtof dataset

## Pairwise spectral similarity

In [ ]:
# This step is performed by "../src/hqtof_pair.py"

## Pairwise chemical similarity

In [ ]:
# Create FP database
IDX2SMILE = []
for idx, SPEC in tqdm(enumerate(Hqtof_CCMSIDs), total=len(Hqtof_CCMSIDs)):
    CCMSID = Hqtof_CCMSIDs[idx]
    SMILE = GNPS_INFO[CCMSID]['SMILE']
    IDX2SMILE.append([SMILE, idx])

FPDB = f'../msdb/data/hqtof/chem/hqtof_fp_db.h5'
if not os.path.exists(FPDB):
    create_db_file(
        mols_source=IDX2SMILE,
        filename=FPDB,
        mol_format='smiles',  # required
        fp_type='Morgan',
        fp_params={'radius': 2, 'fpSize': 1024}
    )

In [ ]:
# FP_SPECTRA search against FS_E_LIB
fpe = FPSim2Engine(fp_filename=FPDB)
hit_dict = {}
for idx in trange(len(Hqtof_CCMSIDs)):
    try:
        CCMSID = Hqtof_CCMSIDs[idx]
        SMILE = GNPS_INFO[CCMSID]['SMILE']
        results = fpe.similarity(SMILE, threshold=0, metric='dice', n_workers=1)  # [(LIBidx,similarity1),(),...]
        results = [x for x in results if x[0] != idx] # Remove self comparison
        fpidx_list = [x[0] for x in results]
        fpsim_list = [x[1] for x in results]

        if fpidx_list:
            hit_dict[idx] = {'idx': fpidx_list, 'sim_list': fpsim_list}  # {LIBidx: {},{},..}
    except:print(idx)

In [ ]:
ECHEM_MATRIX  = np.full((len(Hqtof_CCMSIDs),len(Hqtof_CCMSIDs)), 0.0, dtype=float) 
for key, value in hit_dict.items():
    QueryIdx = key
    MatchIdxs = value['idx']
    MatchSim = value['sim_list']
    for idx, MatchIdx in enumerate(MatchIdxs):
        ECHEM_MATRIX[QueryIdx,MatchIdx] = MatchSim[idx]

# Save results
DIR = '../msdb/data/hqtof/chem/'
np.save(os.path.join(DIR,f'hqtof_dice.npy'),ECHEM_MATRIX)

## Evaluation
TP, FP, TN, FN, ROC, FDR

### ROC curve
Add class distribution
In sklearn.metrices: TN:`C_{0,0}`, FN:`C_{1,0}`, TP: `C_{1,1}` and FP: `C_{0,1}`.

In [ ]:
# Load Data and parameters
# Parameters 
CHEM_THRESHOLD = 0.7
SPEC_THRESHOLD = 0.7 # Thresholds
PEAK_THRESHOLD = 6

ALGORITHMs = G1 + G2 + G3
# ALGORITHMs = ['peak_percentage']

In [ ]:
ALGORITHMs

In [ ]:
# ROC curve without shared peak matrix
# Save roc_auc curve 
plt.figure(figsize=(6, 6), constrained_layout=True)
AUC_DICT = {}
for ALGORITHM in tqdm(ALGORITHMs):
    # try:
        SPEC_SIM_MATRIX = np.load(f'../msdb/data/matrix/{ALGORITHM}_similarity_matrix.npy')
        LABELs,SPEC_SIMILARITYs = [],[]
        TEMP_INFO = {}
        for idx1 in range(len(IDXtoCCMSID)):    
            for idx2 in range(idx1):
                CHEM_SIM = TANIMOTO_MATRIX[idx1,idx2]
                SPEC_SIM = SPEC_SIM_MATRIX[idx1,idx2]
                SPEC_SIMILARITYs.append(SPEC_SIM)
                
                if CHEM_SIM >= CHEM_THRESHOLD:
                    LABELs.append(1)
                else:
                    LABELs.append(-1)
                    
        FPRs,TPRs, THRESHOLDs = roc_curve(LABELs, SPEC_SIMILARITYs)
        AUC = auc(FPRs,TPRs)
        YOUDEN = [TPRs[i] - FPRs[i] for i in range(len(THRESHOLDs))]
        OPTIMAL_THRESHOLD = THRESHOLDs[YOUDEN.index(max(YOUDEN))] # Optimal threshold
        AUC_DICT[f'{ALGORITHM}'] = {'AUC':AUC,'OT':OPTIMAL_THRESHOLD}
        if ALGORITHM == 'modifeid_cosine':
            plt.plot(FPRs, TPRs, color="red",alpha = 0.7, lw= 1.5, label=f"ROC curve (area = {AUC:.3f})")
        if ALGORITHM == 'entropy':
            plt.plot(FPRs, TPRs, color="blue",alpha = 0.7, lw = 1.5, label=f"ROC curve (area = {AUC:.3f})")            
        else:
            plt.plot(FPRs, TPRs, color="grey",alpha = 0.7, lw = 0.8, label=f"ROC curve (area = {AUC:.3f})")
    # except:print(ALGORITHM)
        

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel("False Positive Rate", fontname='Arial', fontsize=12)
plt.ylabel("True Positive Rate", fontname='Arial', fontsize=12)
plt.tick_params(axis='both', which='major', labelsize=12,
                labelfontfamily ='Arial', labelcolor='black', width=1, length=2)
plt.gca().set_aspect('equal', adjustable='box')

# plt.show()

# plt.title("Receiver Operating Characteristic")
# plt.plot([0, 1], [0, 1], color="navy", lw=1, linestyle="--")
# plt.savefig("../msdb/data/pic/ROC_AUC.svg", format="svg", bbox_inches="tight")

plt.savefig("../msdb/data/pic/ROC_AUC.png", dpi=300, bbox_inches="tight")

with open("../msdb/data/pic/ROC_AUC.json",'w') as f: # Save ROC_AUC json
    json.dump(AUC_DICT,f)

### FDR curve

In [ ]:
N_MS1MATCH = 0
for idx1 in trange(len(IDXtoCCMSID)):    
    for idx2 in range(idx1):
        if MS1_MATCH_MATRIX[idx1,idx2] == 1:
            N_MS1MATCH += 1 
print(N_MS1MATCH)

In [ ]:
# FDR v.s. similarity plotting
FDR_DICT = {}
for ALGORITHM in tqdm(['modified_cosine']):#tqdm(ALGORITHMs[:1]):
    try:
        SPEC_SIM_MATRIX = np.load(f'../msdb/data/matrix/{ALGORITHM}_similarity_matrix.npy')
        LABELs,SPEC_SIMILARITYs = [],[]
        TEMP_INFO = {}
        for idx1 in range(len(IDXtoCCMSID)):    
            for idx2 in range(idx1):
                CHEM_SIM = TANIMOTO_MATRIX[idx1,idx2]
                SPEC_SIM = SPEC_SIM_MATRIX[idx1,idx2]          
                
                SPEC_SIMILARITYs.append(SPEC_SIM)
                if CHEM_SIM >= CHEM_THRESHOLD:
                    LABELs.append(1)
                else:
                    LABELs.append(-1)
                    
        FPRs,TPRs, THRESHOLDs = roc_curve(LABELs, SPEC_SIMILARITYs)
        FDRs = [] # Calculate FDR
        for THRESHOLD in tqdm(THRESHOLDs,total=len(THRESHOLDs)):
            Y_PREDCTIONs = []
            for idx1 in range(len(IDXtoCCMSID)):
                for idx2 in range(idx1):
                    SPEC_SIM = SPEC_SIM_MATRIX[idx1,idx2]
                    MS1_MATCH = MS1_MATCH_MATRIX[idx1,idx2]
                    if SPEC_SIM >= THRESHOLD and MS1_MATCH == 1:
                        Y_PREDCTIONs.append(1)
                    else: 
                        Y_PREDCTIONs.append(-1)
            CONFUSION_METRIX = confusion_matrix(LABELs, Y_PREDCTIONs)  
            
            TP,FP = CONFUSION_METRIX[1,1],CONFUSION_METRIX[0,1]
            if (TP + FP) == 0:
                FDRs.append(0) 
            else:
                FDRs.append(FP/(TP+FP))

        plt.plot(THRESHOLDs,FDRs,'b*-',label = f'ROC curve of {ALGORITHM}')
    except:print(ALGORITHM)
        
# plt.plot([0,1],[0,1],'r--',label = '45° reference')
plt.legend()
plt.xlabel('Spectral similarity score')
plt.ylabel('False-discovery rate')
plt.show()

### Spectral similariry distribution compared to modified cosine

In [ ]:
# Load spectral similarity matrices
G1,G2,G3 = evaluation.get_groups()
TALGOs = G1 + G2
TALGOs.remove('modified_cosine')

# ALGORITHMs.insert(0, 'cosine')
# ALGORITHMs.insert(0, 'modified_cosine')

In [ ]:
# 1D hist
DIR = f'../msdb/data/hqtof/SpecSimMatrix/'
ALGORITHMs = ['cosine']
for ALGO in ALGORITHMs:
    plt.figure(figsize=(6, 1.2), constrained_layout=True)
    
    SIMILARITY = [] # Hist data    
    SPEC_SIM_MATRIX = np.load(os.path.join(DIR,f'{ALGO}.npy'))
    for idx1 in range(len(SPEC_SIM_MATRIX)):
        for idx2 in range(idx1):
            SIMILARITY.append(SPEC_SIM_MATRIX[idx1,idx2])
    sns.kdeplot(
        data=SIMILARITY,
        # x=xlabel,
        lw =0.5,
        clip=(0, 1),
        legend=True,
        color="black",
        fill=True,
        bw_method= 0.2,
        warn_singular=False
        )
    plt.axis('off')
    # plt.xticks([])
    # plt.yticks([])
    plt.savefig(f'../msdb/data/pic/hist/{ALGO}_hist.png', dpi=300, bbox_inches= 'tight',pad_inches=0.01)
    plt.show()

In [ ]:
# Load spectral similarity matrices
# Param

DIR = f'../msdb/data/hqtof/' # former: f'../msdb/data/hqtof/matrix/'
ALGORITHMs = ['modified_cosine']
OTHERs = [a for a in TALGOs if a not in ALGORITHMs]
OTHERs = ['cosine']
BINs = 75

for idx1 in range(len(OTHERs)):
    ALGORITHM1 = OTHERs[idx1]
    SPEC_SIM_MATRIX1 = np.load(os.path.join(DIR,f'SpecSimMatrix/{ALGORITHM1}.npy'))
    SIMILARITY1 = []
    for i in range(len(SPEC_SIM_MATRIX1)):
        for j in range(i):
            SIMILARITY1.append(SPEC_SIM_MATRIX1[i,j])
    
    for idx2 in range(len(ALGORITHMs)):
        ALGORITHM2 = ALGORITHMs[idx2]
        SPEC_SIM_MATRIX2 = np.load(os.path.join(DIR,f'SpecSimMatrix/{ALGORITHM2}.npy'))
        SIMILARITY2 = []
        for i in range(len(SPEC_SIM_MATRIX2)):
            for j in range(i):
                SIMILARITY2.append(SPEC_SIM_MATRIX2[i,j])
                
        
        tick_locators = mticker.FixedLocator(np.arange(0, BINs + 1, BINs / 4))
        tick_labels = np.asarray([f"{a:.2f}" for a in np.arange(0, 1.01, 0.25)])
        
        fig, axe = plt.subplots(constrained_layout=True,figsize=(6, 6))
        
        hist, _, _ = np.histogram2d(
            SIMILARITY1, # X label
            SIMILARITY2, # Y label
            bins = BINs,
            range=[[0, 1], [0, 1]],
        )
        hist /= len(SIMILARITY1)
        heatmap = sns.heatmap(
            np.rot90(hist),
            vmin=0.0,
            vmax=0.001,
            cmap="viridis",
            cbar = i == 2,
            cbar_ax = cbar_ax if i == 2 else None,
            # cbar=True, # Legend 
            cbar_kws={"format": mticker.StrMethodFormatter("{x:.3%}")},
            square=True,
            xticklabels=False,
            yticklabels=False,
            norm=LogNorm(vmax=0.001),
            ax = axe
        )
        
        # # axe.plot([0, hist.shape[0]], [hist.shape[0] , 0], color='red', linewidth=1,linestyle="--")
        axe.yaxis.set_major_locator(tick_locators)
        axe.xaxis.set_major_locator(tick_locators)
        axe.set_yticklabels(tick_labels[::-1]) # Reverse
        axe.set_xticklabels(tick_labels)
        axe.set_xlabel(ALGORITHM1, fontname='Arial', fontsize=12)
        axe.set_ylabel(ALGORITHM2, fontname='Arial', fontsize=12)
        
        axe.tick_params(axis='both', which='major', labelsize=12, labelfontfamily ='Arial', width=1, length=5) # Font settings
        # axe.xaxis.set_major_formatter(mticker.PercentFormatter())
        for _, spine in heatmap.spines.items():
            spine.set_visible(True)

        
        axe.plot(
            [0, BINs], [BINs, 0], color="red", linestyle="dashed"
        )
        
        sns.despine(ax=axe) # Remove top and right border
        
    
        # plt.show()
        plt.savefig(os.path.join(DIR,f'2Dhist/{ALGORITHM2}Y_{ALGORITHM1}X_2Dhist.png'), dpi=500, bbox_inches= 'tight',pad_inches=0.5)
        # plt.close()
        

In [ ]:
# Locate specific 
DIR = '../msdb/data/hqtof/'
ALGORITHM1 = 'modified_cosine'
MATRIX1 = np.load(os.path.join(DIR,f'SpecSimMatrix/{ALGORITHM1}.npy'))

G1,G2,G3 = evaluation.get_groups()
TAs = G1+G2+G3
TAs = ['dot_product_reverse']
for ALGORITHM2 in TAs:
    # ALGORITHM2 = 'entropy' #ALGORITHMs[0]
    MATRIX2 = np.load(os.path.join(DIR,f'SpecSimMatrix/{ALGORITHM2}.npy'))
    
    mask = np.triu_indices_from(MATRIX1, k=0) # The upper triangle without diagonal
    count = np.sum(MATRIX2[mask] > MATRIX1[mask]) # 比较 MATRIX2 > MATRIX1 的位置
    
    n = MATRIX1.shape[0]
    upper_indices = np.triu_indices(n, k=0) # The upper triangle without diagonal
    total_count = len(upper_indices[0])
    
    pp = round(count/total_count,2)*100
    
    print(f'{ALGORITHM2} socore higher than {ALGORITHM1} in {pp}% spectral pairs.')

### Spectral and chemical similarity correlation
Violin plot

In [ ]:
G1,G2,G3 = evaluation.get_groups()
ALGOs = G1+G2+G3
# ALGORITHM = [a for a in ALGORITHMs if 'modified_cosine' in a][0]

In [ ]:
 # Load spec matrix
DIR = '../msdb/data/hqtof/matrix/'
ALGO = 'dice'
for ALGO in tqdm(ALGOs,total=len(ALGOs)):
    try:
        ChemSimMat = np.load('../msdb/data/hqtof/chem/ChemSim.npy')
        SpecSimMat = np.load(os.path.join(DIR,f'{ALGO}_similarity_matrix.npy'))
        
        # High sim pairs
        HSIdx = [np.where(arr >=0.7) for arr in ChemSimMat] # [(),(),...] list containing tuples
        HSSpecSim = [SpecSimMat[idx][HSIdx[idx]] for idx in range(len(HSIdx)) if HSIdx[idx][0].any()]
        # HSChemSim = [ChemSimMat[idx][HSIdx[idx]] for idx in range(len(HSIdx)) if HSIdx[idx][0].any()] 
        
        HS_values = np.concatenate(HSSpecSim)
        
        # Low sim pairs
        LSIdx = [np.where((arr >= 0.1) & (arr < 0.35)) for arr in ECHEM_MATRIX] 
        LSSpecSim = [SpecSimMat[idx][LSIdx[idx]] for idx in range(len(LSIdx)) if LSIdx[idx][0].any()]
        LSChemSim = [ChemSimMat[idx][LSIdx[idx]] for idx in range(len(LSIdx)) if LSIdx[idx][0].any()] 
        
        LS_values = np.concatenate(LSSpecSim)
        
        
        
        data_ls = pd.DataFrame({'Values': LS_values, 'Type': 'LS'})
        data_hs = pd.DataFrame({'Values': HS_values, 'Type': 'HS'})
        
        # 创建一个共享 x 轴的双 y 轴图
        fig, ax1 = plt.subplots(figsize=(3, 6))
        
        # 绘制 LS_values 的小提琴图
        sns.violinplot(x='Type', y='Values', data=data_ls, ax=ax1, color='blue', width=0.5,inner=None)
        ax1.set_title(f"{ALGO}")
        ax1.set_ylabel("LS Values", color='blue')
        ax1.tick_params(axis='y', labelcolor='blue')
        
        # 隐藏 ax1 的边框
        for spine in ax1.spines.values():
            spine.set_visible(False)
        ax1.xaxis.set_ticks_position('none')  # 隐藏 x 轴刻度
        ax1.yaxis.set_ticks_position('none')  # 隐藏 y 轴刻度
        
        ax2 = ax1.twinx() # 创建第二个 y 轴
        
        ax2.spines['right'].set_position(('outward', -3)) # 调整 ax2 的位置，使其更靠近 ax1
        
        # 绘制 HS_values 的小提琴图
        sns.violinplot(x='Type', y='Values', data=data_hs, ax=ax2, color='red', width=0.5,inner=None)
        ax2.set_ylabel("HS Values", color='red')
        ax2.tick_params(axis='y', labelcolor='red')
        
        
        # 隐藏 ax2 的边框
        for spine in ax2.spines.values():
            spine.set_visible(False)
        
        # 调整布局
        plt.tight_layout()
        plt.savefig(f'../msdb/data/hqtof/pic_violin/{ALGO}.png', dpi=500, bbox_inches="tight")
        plt.close()
    except:print(ALGO)

In [ ]:
# View
print(FS_SPECTRA[1]['smile'])
print(FS_SPECTRA[113]['smile'])

In [ ]:
FS_SPECTRA[113]['peaks']

## Detailed inspection

In [3]:
Hqtof_CCMSIDs = list(np.load('../msdb/data/hqtof/idlist/H_qtof_non-redundant_CCMSIDs.npy'))
# Load processed json file
GNPS_INFO = functions_new.json_load('../msdb/GNPSLIBRARY/GNPS-LIBRARY-INFO.json')

modified_cosine = ModifiedCosine(tolerance = 0.05)
neutral_loss = NeutralLossesCosine(tolerance = 0.05)
cosine = CosineGreedy(tolerance=0.05)

In [4]:
# optimial threholds
optimal_threshold = {'modified_cosine': 0.72, 'weighted_dot_product': 0.89, 'hellinger': 0.18, 'ms_for_id': 0.205, 'manhattan': 0.535,
            'absolute_value': 0.535, 'intersection': 0.535, 'lorentzian': 0.52, 'ruzicka': 0.365, 'motyka': 0.535, 'fidelity': 0.72,
            'bhattacharya_2': 0.755, 'matusita': 0.475, 'bhattacharya_1': 0.765, 'squared_chord': 0.72, 'vicis_symmetric_chi_squared_3': 0.625,
            'probabilistic_symmetric_chi_squared': 0.67, 'harmonic_mean': 0.67, 'unweighted_entropy': 0.700, 'improved_similarity': 0.515,
            'clark': 0.515, 'spectral_contrast_angle': 0.72, 'pearson_correlation': 0.845, 'dot_product': 0.72, 'inner_product': 0.1,
            'whittaker_index_of_association': 0.11, 'dot_product_reverse': 0.99, 'avg_l': 0.175, 'entropy': 0.535, 'roberts': 0.285,
            'baroni_urbani_buser': 0.735, 'neutral_loss': 0.72, 'jaccard': 0.54, 'dice': 0.52, 'peak_percentage': 0.71, 'squared_euclidean': 0.975,
            'euclidean': 0.84, 'penrose_shape': 0.84, 'chebyshev': 0.885, 'ms_for_id_v1': 0.98, 'canberra': 0.055, 'divergence': 0.03, 'wave_hedges': 0.055,
            'penrose_size': 0.23, 'mean_character': 0.985, 'symmetric_chi_squared': 0.950}

In [26]:
DIR = '../msdb/data/hqtof'
ALGORITHM1 = 'modified_cosine'
MATRIX1 = np.load(os.path.join(DIR,f'SpecSimMatrix/{ALGORITHM1}.npy'))

ALGORITHM2 = 'peak_percentage'
MATRIX2 = np.load(os.path.join(DIR,f'SpecSimMatrix/{ALGORITHM2}.npy'))

ChemSimMat = np.load(os.path.join(DIR,f'chem/hqtof_dice.npy'))

# Get part of score higher
i, j = np.triu_indices_from(MATRIX1, k=1)  # The upper triangle without diagonal
mask_condition = MATRIX2[i, j] > MATRIX1[i, j]
indices = (i[mask_condition], j[mask_condition])
pair_indices = list(zip(indices[0],indices[1]))


mask_condition1 = MATRIX2[i, j] < MATRIX1[i, j] # Mask where MATRIX2 is less than MATRIX1
less_than_indices = (i[mask_condition1], j[mask_condition1]) # Use the mask to get the relevant indices
less_than_pair_indices = list(zip(less_than_indices[0], less_than_indices[1])) # Create a list of paired indices


TPs, FPs, TNs, FNs = [],[],[],[]
for i in tqdm(less_than_pair_indices,total = len(less_than_pair_indices)):
    idx1,idx2 = i
    SIM1 = MATRIX1[idx1,idx2]
    SIM2 = MATRIX2[idx1,idx2]
    diff = abs(SIM1-SIM2)
    ChemSim = ChemSimMat[idx1,idx2]
    OT = optimal_threshold[ALGORITHM1]
    # if diff >= 0: # Filter by similarity difference
    
    if SIM1 > 0 and SIM2 >= OT and ChemSim >= 0.75:
        TPs.append(i)
    elif SIM1 > 0 and SIM2 >= OT and ChemSim < 0.75:
        FPs.append(i)
    elif SIM1 > 0 and SIM2 < OT and ChemSim >= 0.75:
        FNs.append(i)
    elif SIM1 > 0 and SIM2 < OT and ChemSim < 0.75:
        TNs.append(i)
len(TPs),len(FPs),len(TNs),len(FNs)

(2571, 2571)


In [ ]:
PAIR = TPs[0]
idx1, idx2 = PAIR[0],PAIR[1]
CCMSID1,CCMSID2 = Hqtof_CCMSIDs[idx1], Hqtof_CCMSIDs[idx2]
SPEC1, SPECTRUM1 = functions_new.gnps_info_format(GNPS_INFO,CCMSID1)
SPEC2, SPECTRUM2 = functions_new.gnps_info_format(GNPS_INFO,CCMSID2)
SIM1,MP1 = functions_new.clac_spec_sim(SPEC1, SPECTRUM1,SPEC2, SPECTRUM2,ALGORITHM1)
SIM2,MP2 = functions_new.clac_spec_sim(SPEC1, SPECTRUM1,SPEC2, SPECTRUM2, ALGORITHM2)
PM1, PM2 = float(GNPS_INFO[CCMSID1]["PEPMASS"]),float(GNPS_INFO[CCMSID2]["PEPMASS"])
print(f'Top {CCMSID1}, precursor {PM1}')
print(f'Bottom {CCMSID2}, precursor {PM2}')
print(f"Re-calced {ALGORITHM1}:{round(SIM1,2)}",f"{ALGORITHM2}: {round(SIM2,2)}")

SMILE1 = GNPS_INFO[CCMSID1]['SMILE']
SMILE2 = GNPS_INFO[CCMSID2]['SMILE']
print(f'Matched peaks',MP1,MP2,f'Chemical dice {round(cheminfo_tools.dice(SMILE1,SMILE2),2)}')
print(CCMSID1,MATRIX1[idx1,idx2], CCMSID2,MATRIX2[idx1,idx2]) # 

print(f"{SMILE1}\n{SMILE2}")

In [ ]:
# To output matched peaks and scored by scripts
spectrum_1 = sus.MsmsSpectrum(identifier='1', 
                              precursor_mz=PM1, 
                              precursor_charge=1,
                              mz=np.array(SPEC1[:, 0],dtype=np.float64),
                              intensity=np.array(SPEC1[:, 1],dtype=np.float64))
spectrum_2 = sus.MsmsSpectrum(identifier='2', 
                              precursor_mz=PM2, 
                              precursor_charge=1,
                              mz=np.array(SPEC2[:, 0],dtype=np.float64),
                              intensity=np.array(SPEC2[:, 1],dtype=np.float64)) 
MCS_RES = peaktools.modified_cosine(spectrum_1,spectrum_2,0.1)
NL_RES = peaktools.neutral_loss(spectrum_1,spectrum_2,0.1)
MCS_RES,NL_RES

In [ ]:
# MCS matched peaks and its indices
matched_indices = MCS_RES.matched_indices
matched_indices_other = MCS_RES.matched_indices_other
print(f'Shared peaks of mcs in spectrum1:{spectrum_1.mz[matched_indices]}')
print(f'Shared peaks of mcs in spectrum2:{spectrum_2.mz[matched_indices]}')

# NL shared peaks and its indices
nl_spectrum_1 = peaktools.spec_to_neutral_loss(spectrum_1)
nl_spectrum_2 = peaktools.spec_to_neutral_loss(spectrum_2)
nl_matched_indices = NL_RES.matched_indices
nl_matched_indices_other = NL_RES.matched_indices_other
PM1-PM2,nl_spectrum_1.mz[nl_matched_indices],nl_spectrum_2.mz[nl_matched_indices_other]

In [ ]:
nl_spectrum = nl_spectrum_2
np.column_stack((nl_spectrum.mz,nl_spectrum.intensity))

In [ ]:
# Matchms v
score1 = modified_cosine.pair(SPECTRUM1,SPECTRUM2)
score2 = neutral_loss.pair(SPECTRUM1,SPECTRUM2)
score1,score2